<a href="https://colab.research.google.com/github/msaad1311/Machine-Translation-English-to-French/blob/master/Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import collections
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


In [0]:
with open('/content/europarl-v7.fr-en.en', "r") as f:
        data = f.read()

english_sentences = data.split('\n')

with open('/content/europarl-v7.fr-en.fr', "r") as f:
        data = f.read()

french_sentences = data.split('\n')

In [0]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  Resumption of the session
small_vocab_fr Line 1:  Reprise de la session
small_vocab_en Line 2:  I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
small_vocab_fr Line 2:  Je déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances.


In [0]:
#English Preprocessing

en_tlk = Tokenizer()
en_tlk.fit_on_texts(english_sentences)
entk = en_tlk.texts_to_sequences(english_sentences)
en_pad = pad_sequences(entk, maxlen=len(max(entk,key=len)), padding='post')

In [0]:
#French Preprocessing

fr_tlk = Tokenizer()
fr_tlk.fit_on_texts(french_sentences)
frtk=fr_tlk.texts_to_sequences(french_sentences)
fr_pad = pad_sequences(frtk,maxlen=len(max(frtk,key=len)),padding='post')

In [0]:
fr_pad = fr_pad.reshape(*fr_pad.shape, 1)

NameError: ignored

In [0]:
max_english_sequence_length = entk.shape[1]
max_french_sequence_length = frtk.shape[1]
english_vocab_size = len(entlk.word_index)
french_vocab_size = len(frtlk.word_index)

In [0]:
print(max_english_sequence_length)
print(max_french_sequence_length)
print(english_vocab_size)
print(french_vocab_size)